In [ ]:
# Final code to be used

import time

# Record the start time
start_time = time.time()

import os
import pandas as pd
import numpy as np
import math
from datetime import date

folder_path = input("Please input the directory to the files -> /Users/jiayikoh/Downloads/07 GI Commission 3/\n")
while os.path.isdir(folder_path) != True:
    folder_path = input("Directory does not exist, please re-enter the directory to the files -> /Users/jiayikoh/Downloads/07 GI Commission 3/\n")
os.chdir(folder_path)

# read AIG comm statement

file_pattern = '01*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

aig_1 = pd.read_excel(matching_files[0], sheet_name='Commission_Final', skipfooter=1)
aig_2 = pd.read_excel(matching_files[0], sheet_name='Sheet1', header=1, usecols=[1, 2], skipfooter=1)


# name the GST Type column
aig_2 = aig_2.rename(columns={'Unnamed: 2': 'GST Type'})

# merge both dfs
aig = pd.concat([aig_1, aig_2[['GST Type']]], axis=1)

# remove "IPPFA - " from the ADVISER column (so that it could improve the FAR names matching algorithms)
aig['ADVISER'] = aig['ADVISER'].str.replace('IPPFA - ', '')

# rename the columns as per the working file
columns = ['ADVISER', 'POLICY/ENDT', 'POLICY EFF DATE', 'DESCRIPTION/PARTICULARS', 'COMM AMT', 'GST ON COMM']

rename_col = ['TFAR', 'Policy no.', 'Pol Date', 'Insured Name', 'Comm.Recd (with GST)', 'GST amt']

label = dict(zip(columns, rename_col))

aig = aig.rename(columns=label)

# create "Insurer" column
aig['Insurer'] = 'AIG-GI'

# read AIA comm statement
file_pattern = '03*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

aia = pd.read_excel(matching_files[0], header=2, skipfooter=3)

# forward fill the NaN with the previous valid observation
aia = aia.fillna(method='ffill')

# rename the columns as per the working file
columns = ['TFAR Name', 'polno', 'Sum of TOTAL AMOUNT', 'Sum of GST']

rename_col = ['TFAR', 'Policy no.', 'Comm.Recd (with GST)', 'GST amt']

label = dict(zip(columns, rename_col))

aia = aia.rename(columns=label)

# create "Insurer" column
aia['Insurer'] = 'AIA-GI'

# Allianz Insurance commission statement
file_pattern = '04*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

allianz = pd.read_excel(matching_files[0], skipfooter=1)

# rename the columns as per the working file
columns = ['Policy Number', 'Agent Name', 'Effective Date', 'Total Commission', 'GST on Commission']

rename_col = ['Policy no.', 'TFAR', 'Pol Date', 'Comm.Recd (with GST)', 'GST amt']

label = dict(zip(columns, rename_col))

allianz = allianz.rename(columns=label)

# create "Insurer" column
allianz['Insurer'] = 'ALLIANZ-GI'

# Allied World commission statement
allied = pd.read_excel('/Users/jiayikoh/Downloads/07 GI Commission 2/Allied World commission statement.xlsx', header=6, skipfooter=13)

# remove NaN in Policy No.
allied = allied.dropna(subset=['Commission'])

# forward fill NaN
allied.loc[:, ['Account No.', 'Account Name', 'Currency', 'Payable']] = allied.loc[:, ['Account No.', 'Account Name', 'Currency', 'Payable']].fillna(method='ffill')

# remove 'Unnamed' columns
allied = allied.loc[:, ~allied.columns.str.contains('Unnamed')]

# rename the columns as per the working file
columns = ['Policy No.', 'TFAR Name', 'Commission GST']

rename_col = ['Policy no.', 'TFAR', 'GST amt']

label = dict(zip(columns, rename_col))

allied = allied.rename(columns=label)

# create 'Insurer' column
allied['Insurer'] = 'ALLIED WORLD-GI'
allied['Comm.Recd (with GST)'] = allied[['Commission', 'GST amt']].sum(axis=1)*(-1)


# Chubb commission statement
file_pattern = '09*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

chubb = pd.read_excel(matching_files[0], header=7, skipfooter=1)

# remove 'Unnamed' columns
chubb = chubb.loc[:, ~chubb.columns.str.contains('Unnamed')]

# rename the columns as per the working file
columns = ['Policy No.', 'Agent', 'Comm']

rename_col = ['Policy no.', 'TFAR', 'Comm.Recd (with GST)']

label = dict(zip(columns, rename_col))

chubb = chubb.rename(columns=label)

# create 'Insurer' column
chubb['Insurer'] = 'CHUBB-GI'

# FWD commission statement
file_pattern = '15*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

fwd_adviser = pd.read_excel(folder_path + 'FWD Adviser Codes List.xlsx')
fwd = pd.read_excel(matching_files[0])

fwd['agent_id_number'] = pd.to_numeric(fwd['agent_id_number'], errors='coerce')

# merge with fwd advisor code list for the adviser details
fwd = pd.merge(fwd, fwd_adviser, how='left', left_on='agent_id_number', right_on='FWD Life code')

# rename the columns as per the working file
columns = ['policy_number', 'GI Advisers', '$ txn commission', '$ txn gst commission']

rename_col = ['Policy no.', 'TFAR', 'Comm.Recd (with GST)', 'GST amt']

label = dict(zip(columns, rename_col))

fwd = fwd.rename(columns=label)

# create 'Insurer' column
fwd['Insurer'] = 'FWD-GI'

# Great Eastern commission statement
file_pattern = '17*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

ge = pd.read_excel(matching_files[0], sheet_name='MAY 23', skipfooter=2)

# rename the columns as per the working file
columns = ['Policy Number', 'Agent Name', 'Total Net Amount in Accounting Currency']

rename_col = ['Policy no.', 'TFAR', 'Comm.Recd (with GST)']

label = dict(zip(columns, rename_col))

ge = ge.rename(columns=label)

# create 'Insurer' column
ge['Insurer'] = 'GE-GI'


# HSBC commission statement
file_pattern = '20_1*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

hsbc_1 = pd.read_excel(matching_files[0], skipfooter=1)

file_pattern = '20_2*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

hsbc_2 = pd.read_excel(matching_files[0], sheet_name='Detailed Breakdown (Earned)', header=9, skipfooter=2)

# rename the columns as per the working file
columns = ['POLNUM', 'RIDESC', 'COMM_LCEAMT']

rename_col = ['Policy no.', 'TFAR', 'Comm.Recd (with GST)']

label = dict(zip(columns, rename_col))

hsbc_1 = hsbc_1.rename(columns=label)

# create 'Insurer' column
hsbc_1['Insurer'] = 'HSBC-GI'


# rename the columns as per the working file
columns = ['Policy No.', 'Commission Amount']

rename_col = ['Policy no.', 'Comm.Recd (with GST)']

label = dict(zip(columns, rename_col))

hsbc_2 = hsbc_2.rename(columns=label)

# create 'Insurer' column
hsbc_2['Insurer'] = 'HSBC-GI'

hsbc = pd.concat([hsbc_1, hsbc_2])


# Liberty commission statement
file_pattern = '23*.csv*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

liberty_adviser = pd.read_excel(folder_path + 'Liberty Adviser codes list.xlsx')
liberty = pd.read_csv(matching_files[0])

# merge with liberty adviser code list for the adviser details
liberty = pd.merge(liberty, liberty_adviser, how='left', left_on='Sub Agent Code', right_on='CODE')

# rename the columns as per the working file
columns = ['Policy/Renewal/Endorsement', 'NAME OF ADVISER', 'Name of Insured', 'Total Commission Paid', 'Commission GST']

rename_col = ['Policy no.', 'TFAR', 'Insured Name', 'Comm.Recd (with GST)', 'GST amt']

label = dict(zip(columns, rename_col))

liberty = liberty.rename(columns=label)

# create 'Insurer' column
liberty['Insurer'] = 'LIBERTY-GI'

# MSIG commission statement

file_pattern = '25*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

msig = pd.read_excel(matching_files[0], header=32)
msig_soa = pd.read_excel(folder_path + 'MSIG SOA.xlsx', header=7)

# drop rows where all column values are NaN
msig = msig.dropna(how='all')

# drop columns where all row values are NaN
msig = msig.dropna(how='all', axis=1)

# filter without 'Settlement Date' and with 'Unnamed: 27' - commission amount
msig = msig[(msig['Settlement Date'].isna()) & (msig['Unnamed: 27'].notna())]

# merge commission statement and SOA
msig = pd.merge(msig, msig_soa[['Name of FA Rep', 'Policy Number']], how='left', left_on='Policy No\n', right_on='Policy Number')

# rename the columns as per the working file
columns = ['Policy No\n', 'Name of FA Rep', 'Unnamed: 37', 'Unnamed: 33']

rename_col = ['Policy no.', 'TFAR', 'Comm.Recd (with GST)', 'GST amt']

label = dict(zip(columns, rename_col))

msig = msig.rename(columns=label)

# create 'Insurer' column
msig['Insurer'] = 'MSIG-GI'


# QBE commission statement

file_pattern = '28*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

qbe_adviser = pd.read_excel(folder_path + 'QBE-Qnect User List.xlsx')
qbe = pd.read_excel(matching_files[0], skipfooter=1)

# remove whitespace in the agent code
qbe_adviser['P400_USER'] = qbe_adviser['P400_USER'].str.strip()

# merge with QBE adviser code list to get adviser details
qbe = pd.merge(qbe, qbe_adviser, how='left', left_on='REP_NAME', right_on='P400_USER')

# rename the columns as per the working file
columns = ['POLICY_NUMBER', 'Agent Name', 'COMMISSION_SGD', 'GST_ON_COMMISSION_SGD']

rename_col = ['Policy no.', 'TFAR', 'Comm.Recd (with GST)', 'GST amt']

label = dict(zip(columns, rename_col))

qbe = qbe.rename(columns=label)

# create 'Insurer' column
qbe['Insurer'] = 'QBE-GI'


# Singlife commission statement

file_pattern = '30*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

singlife = pd.read_excel(matching_files[0])

# rename the columns as per the working file
columns = ['POLNUM', 'SRVAGNAME', 'Total', 'GST on Commission']

rename_col = ['Policy no.', 'TFAR', 'Comm.Recd (with GST)', 'GST amt']

label = dict(zip(columns, rename_col))

singlife = singlife.rename(columns=label)

# create 'Insurer' column
singlife['Insurer'] = 'SINGLIFE-GI'


# Sompo commission statement
file_pattern = '31*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

sompo = pd.read_excel(matching_files[0])

# remove columns with all the row values are NaN
sompo = sompo.dropna(how='all', axis=1)

# rename the columns as per the working file
columns = ['Policy No.', 'Producer Name', 'Comm.', 'GST Comm.']

rename_col = ['Policy no.', 'TFAR', 'Comm.Recd (with GST)', 'GST amt']

label = dict(zip(columns, rename_col))

sompo = sompo.rename(columns=label)

# create 'Insurer' column
sompo['Insurer'] = 'QBE-GI'


# get the all the headers of the working file
file_pattern = 'P0*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

working_col = pd.read_excel(matching_files[0], sheet_name='Apr-23', header=3).columns

# create an empty dataframe with the above column names
wk_df = pd.DataFrame(columns=working_col).reset_index()

# combine all processed commission statement df
wk_df = pd.concat([aig, aia, allianz, allied, chubb, fwd, ge, liberty, qbe, singlife, sompo], ignore_index=True)

# read the FAR masterlist
far_master = pd.read_excel('FARs Masterlist 2023.xlsx', skiprows=1)

# merge with itself to get the old agent's manager details
far_master = pd.merge(far_master, far_master[['NEW FAR NAME', 'Manager', 'Mgr Rate']].rename(columns={'Manager':'Manager 2 Name', 'Mgr Rate':'Manager 2 GR%'}),
                     how='left', left_on='FAR (Old)', right_on='NEW FAR NAME').rename(columns={'NEW FAR NAME_x':'NEW FAR NAME'})

# read the revised name list
revised_far_list = pd.read_excel('GI comm name matching.xls')

# find the name in FAR masterlist using the revised name list provided by the name matching algorithm
wk_df = pd.merge(wk_df, revised_far_list[['ADVISER', 'matched_name']], how='left', left_on='TFAR', right_on='ADVISER')

# match the adviser details using the name matching list
wk_df = pd.merge(wk_df, far_master[['NEW FAR NAME', 'Advisory Group', 2023, 'FAR Status']].rename(columns={2023:'GR%'}), how='left', left_on='matched_name', right_on='NEW FAR NAME')

# load the e-submission list for the referrer list
e_sub = pd.read_excel('GI e-submission list.xlsx')

# match policy number to get referrer details
wk_df = pd.merge(wk_df, e_sub[['Policy No', 'Name of Referral']].rename(columns={'Name of Referral':'Referrer'}), how='left', left_on='Policy no.', right_on='Policy No')

# rename the columns as per the working file
columns = ['TRANS CODE', 'ADVISER', 'POLICY/ENDT', 'POLICY EFF DATE', 'DESCRIPTION/PARTICULARS', 'COMM AMT', 'GST ON COMM']

rename_col = ['Payment / Clawback', 'TFAR', 'Policy no.', 'Pol Date', 'Insured Name', 'Comm.Recd (with GST)', 'GST amt']

# get the required fields 
columns = ['Policy no.', 'Insured Name', 'Insurer', 'TFAR', 'Referrer', 'Advisory Group', 'Comm.Recd (with GST)', 'GST amt', 'GR%']

wk_df = wk_df.rename(columns={'Advisory Group_y':'Advisory Group'})[columns].drop_duplicates()

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'S1 - Working {date.today():%Y/%m/%d}.xlsx')

# # Write each dataframe to a different worksheet.
wk_df.to_excel(writer, sheet_name='Data')
#df_1_pivot.to_excel(writer, sheet_name='Pivot')

# # Close the Pandas Excel writer and output the Excel file.
writer.save()


# get all the required fields from the first document
file_pattern = 'P1*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

df_1_col = pd.read_excel(matching_files[0], sheet_name='Data', skiprows=1).columns

file_pattern = 'P0*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

# use the working file df to extract the relevant information for "1-Consolidated GI Com Month"
working_df = pd.read_excel(matching_files[0], sheet_name='Apr-23', header=3)

# select the required fields
df_1 = working_df[df_1_col]


# Create a Pandas Excel writer using XlsxWriter as the engine.
# writer = pd.ExcelWriter('1 - Consolidated GI Com May 2023.xls')
writer = pd.ExcelWriter(f'S2 - 1 - Consolidated GI Com {date.today():%Y/%m/%d}.xlsx')

# # Write each dataframe to a different worksheet.
df_1.to_excel(writer, sheet_name='Data')
# #df_1_pivot.to_excel(writer, sheet_name='Pivot')

# # Close the Pandas Excel writer and output the Excel file.
writer.save()



file_pattern = 'P2*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

df_2_col = pd.read_excel(matching_files, sheet_name='data', skiprows=1).columns

# df_2 contains Paid only policies
df_2 = working_df[working_df['Payment / Clawback'] == 'Paid']

# rename columns to match the required format
df_2 = df_2.rename(columns={'TFAR Name':'FAR Name', 'Total Paid':'Gross Revenue Received', 'GR%':"FAR's GR% Entitlement",
                    'FAR Comm + Referral':'FAR Net Comm'})

# create 'Type' column
df_2['Type'] = 'General Insurance'

# create columns for FAR Receiving Comm, Advisory Group2
df_2['FAR Receiving Comm'] = df_2['FAR Name']
df_2['Advisory Group2'] = df_2['Advisory Group']
df_2["FAR's Status"] = df_2['FAR Receiving Comm Status']

# merge with FAR Masterlist to get the NRIC
df_2 = pd.merge(df_2, far_master[['NEW FAR NAME', 'NRIC']], how='left', left_on='TFAR', right_on='NEW FAR NAME')

# filter to get the required fields
df_2 = df_2[df_2_col]

# Create a Pandas Excel writer using XlsxWriter as the engine.
# writer = pd.ExcelWriter('2 - FAR GI Comm Payout May 2023.xls')
writer = pd.ExcelWriter(f'S3 - 2 - FAR GI Comm Payout {date.today():%Y/%m/%d}.xlsx')

# # Write each dataframe to a different worksheet.
df_2.to_excel(writer, sheet_name='Data')
# #df_2_pivot.to_excel(writer, sheet_name='Pivot')

# # Close the Pandas Excel writer and output the Excel file.
writer.save()


file_pattern = 'P3*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))


df_3_col = pd.read_excel(matching_files[0], header=2).columns

df_3 = working_df.copy()

# merge with FAR Masterlist to get manager details
df_3 = pd.merge(df_3, far_master[['NEW FAR NAME', 'Manager', 'Mgr Rate', 'Manager 2 Name', 'Manager 2 GR%']].rename(columns={'Manager':'Manager Name', 'Mgr Rate':'Manager GR%'}), 
               how='left', left_on='TFAR', right_on='NEW FAR NAME')

df_3 = df_3.rename(columns={'FAR Receiving Comm Status': 'FAR Status', }).drop_duplicates()

df_3 = df_3.reindex(columns=df_3_col, fill_value=None)
df_3['Manager AG'] = np.where(df_3['Manager Name'].isna(), np.nan, df_3['Advisory Group'])
df_3['Manager 2 AG'] = np.where(df_3['Manager 2 Name'].isna(), np.nan, df_3['Advisory Group'])

# # Create a Pandas Excel writer using XlsxWriter as the engine.
# #writer = pd.ExcelWriter('3 - GI Overridding Split May 2023.xls')
writer = pd.ExcelWriter(f'S4 - 3 - GI Overriding Split {date.today():%Y/%m/%d}.xlsx')

# # Write each dataframe to a different worksheet.
df_3.to_excel(writer, sheet_name='Data')

# # Close the Pandas Excel writer and output the Excel file.
writer.save()


# this is for the GI GR file
file_pattern = 'P4*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))


df_4_col = pd.read_excel(matching_files[0]).columns

df_4 = df_2.rename(columns={'Pol #':'Policy No', 'Cash Book #':'Cash Book', 'Carrier':'Insurer', 'FAR Name':'TFAR',
                           })

df_4 = working_df.rename(columns={'Cashbook ref. no.':'Cash Book', 'Policy no.':'Policy No', 'Referrer':'REFERRAL',
                                 'Advisory Group':'GROUP', 'Comm. Recd        (with GST)':'Comm W/O GST',
                                 'GR%':'TFAR %', 'Amt':'TFAR Comm', '%.1':'Referrer %', 'Amt.1':'Referrer Comm'
                                 })

df_4['Adjustment'] = np.nan

df_4 = df_4.reindex(columns=df_4_col, fill_value=None)

# # Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'S5 - GI GR {date.today():%Y/%m/%d}.xlsx')

# # Write each dataframe to a different worksheet.
df_4.to_excel(writer, sheet_name='Data')
# #df_2_pivot.to_excel(writer, sheet_name='Pivot')

# # Close the Pandas Excel writer and output the Excel file.
writer.save()


# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")